# Assignment 02


Recursion for divide and conquer is covered in [chapter 1](https://jeffe.cs.illinois.edu/teaching/algorithms/book/01-recursion.pdf) of Jeff Erickson's book.

Additional readings include

- [Karatsuba's original paper](./OER/karatsuba1962.pdf) from 1962 (in Russian, but the math is almost readable).

- A [1995 reflection by Karatsuba about his technique](./OER/karatsuba1995.pdf). Section 6 describes how the result of $\mathcal O (n^{\log_23})$ became known.

- [A general method for solving divide and conquer recurrences](./OER/Bentley1978.pdf) is the original proof from 1978 of what is known today as a _Master Theorem._


## Divide-and-conquer multiplication

Multiplying large numbers has been, historically, a challenging task for computers. Early computers, operating on 8-bit architectures could handle integers in the interval $[-128. 127]$. In general, we expect a computer with a $b$-bits architecture to handle up to $2^b$ integers; or spread between negative and positive values to cover the interval $[-2^{b-1}, 2^{b-1}-1]$.

Consider two integer numbers $x$ and $y$ with the same number of digits (for example $x=1234$ and $y=5678$). Furthermore, we'll assume that the number of digits these numbers have is a power of 2. Any such number can be written as

$$
\begin{align*}
x & = a10^m + b \\
y & = c10^m + d
\end{align*}
$$

Where $m=\lceil\log_{10} x\rceil/2$. For example, $\lceil\log_{10} 1234\rceil = 4$ and in this case, $m=2$. The _ceiling_ operator $\lceil\cdot\rceil$ in Python is implemented by the function `math.ceil`.

The terms $a,b,c$ and $d$ can be obtained by splitting $x$ and $y$ in left and right halves. This is easy to accomplish when the numbers are represented as strings of digits. For example:


In [1]:
x = "1234"
y = "5678"
m = len(x) // 2
a = x[:m]
b = x[m:]
c = y[:m]
d = y[m:]

String representation is one option for numbers so big that cannot be represented by a languages data types. Another option is to use a list of digits, for example: `x=[1,2,3,4]` and `y=[5,6,7,8]`. Splitting these lists in left and right halves, is also easy to do in python. In fact, the code will be the same as above (except for the first two lines, of course).

Using the split form for $x$ and $y$, their product can be written as

$$
xy = ac10^{2m} + (ad+bc)10^m + bd
$$

The expression above "reduces" the product of two big numbers into four products of smaller numbers. For example,

$$
1234 \times 5678
=
12 \times 56 \times 10^4
+
\left(12 \times 78 + 34 \times 56\right) \times 10^2
+
34 \times 78
$$

We can apply the same process to the smaller products, ending up with single digit multiplications, which are easy to do, and multiplications with powers of 10 which are also easy to do as they only require shifting the number the left and padding with 0 to the right:

$$
\begin{align*}
12 \times 56 & = 1 \times 5 \times 10^2 + (1 \times 6 + 2 \times 5) \times 10 + 2 \times 6 \\
             & = 5 \times 10^2 + 16 \times 10 + 12 \\
             & = 500 + 160 + 12 \\
             & = 672 \\ \\
12 \times 78 & = 1 \times 7 \times 10^2 + (1 \times 8 + 2 \times 7) \times 10 + 2 \times 8 \\
             & = 936 \\ \\
34 \times 56 & = 3 \times 5 \times 10^2 + (3 \times 6 + 4 \times 5) \times 10 + 4 \times 6 \\
             & = 1904 \\ \\
34 \times 78 & = 3 \times 7 \times 10^2 + (3 \times 8 + 4 \times 7) \times 10 + 4 \times 8 \\
             & = 2652
\end{align*}
$$

Substituting these values to the previous expression we get

$$
\begin{align*}
1234 \times 5678 & = 12 \times 56 \times 10^4
                   + \left(12 \times 78 + 34 \times 56\right) \times 10^
                   + 34 \times 78 \\
                 & = 672 \times 10^4 + (936+1904) \times 10^2 + 2652 \\
                 & = 672 \times 10^4 + 2840 \times 10^2 + 2652 \\
                 & = 6720000 +284000 + 2652 \\
                 & = 7006652

\end{align*}
$$

The code below implements this recursive multiplication using strings to represent integer numbers.


In [2]:
def add_strints(x: str, y: str) -> str:
    """
    Add two nonnegative integer strings by converting to int.
    This method can be rewritten as a sum/carry adder for a
    single digit addition, pulling characters from the
    input strings. For simplicity now, we just convert the
    whole string to integer, do the addition, and then
    convert the number back to string.
    """
    return str(int(x) + int(y))


def simple_recursive_multiplication(x: str, y: str) -> str:
    """
    Recursive multiplication for nonnegative integer strings.
    Assumptions:
      - len(x) == len(y)
      - len(x) is a power of two
      - x and y contain only digits
    Uses:
      xy = ac*10^n + (ad+bc)*10^(n/2) + bd
    """
    # Number of digits in x, y
    n = len(x)
    # Base case
    if n == 1:
        return str(int(x) * int(y))
    # Middle of x, y for splitting them in left/right halves
    m = n // 2
    # Divide x, y into left/right halves
    a = x[:m]
    b = x[m:]
    c = y[:m]
    d = y[m:]
    # Compute the partial solution
    ac = simple_recursive_multiplication(a, c)
    ad = simple_recursive_multiplication(a, d)
    bc = simple_recursive_multiplication(b, c)
    bd = simple_recursive_multiplication(b, d)
    # Conquer the partial solutions
    ad_plus_bc = add_strints(ad, bc)
    # Multiply by powers of 10 via appending zeros (string shift).
    term1 = ac + ("0" * n)
    term2 = ad_plus_bc + ("0" * m)
    # Final sum (Using int conversion for addition to keep things simple)
    return str(int(term1) + int(term2) + int(bd))


# --- quick sanity checks ---
if __name__ == "__main__":
    tests = [
        ("12", "34"),
        ("99", "99"),
        ("0123", "0456"),
        ("1234", "5678"),
        ("0000", "0000"),
        ("1111", "0001"),
        ("1234567890123456", "9876543210123456"),
        ("12345678901234561234567890123456", "12345678901234561234567890123456"),
        ("1234567890123456123456789012345612345678901234561234567890123456", "1234567890123456123456789012345612345678901234561234567890123456"),
    ]

    for x, y in tests:
        # Compare against Python int multiplication for correctness.
        got = simple_recursive_multiplication(x, y)
        want = str(int(x) * int(y))
        print(f"{x} * {y} = {got}  (ok={got == want})")

12 * 34 = 408  (ok=True)
99 * 99 = 9801  (ok=True)
0123 * 0456 = 56088  (ok=True)
1234 * 5678 = 7006652  (ok=True)
0000 * 0000 = 0  (ok=True)
1111 * 0001 = 1111  (ok=True)
1234567890123456 * 9876543210123456 = 12193263112635260231976841383936  (ok=True)
12345678901234561234567890123456 * 12345678901234561234567890123456 = 152415787532388203170249644871236061576303002601726870921383936  (ok=True)
1234567890123456123456789012345612345678901234561234567890123456 * 1234567890123456123456789012345612345678901234561234567890123456 = 1524157875323882031702496448712391098920536503657902759142813607364731048132908548544433921658436061576303002601726870921383936  (ok=True)


The technique above divides a problem of size $n$ into four subproblems of size $n/2$. Assembling the partial solutions is done in about $n$ steps (just adding the partial products). Therefore, the recursion pattern is described by:

$$
T(n) = 4T(n/2) + n
$$

And so we have $r=4$, $c=2$, and $d=1$. Therefore, $r>c^d$, and $T(n)\in\mathcal O(n^{\log_cr})$. Here, $\log_cr=\log_24=2$ and so the multiplication technique above performs in $\mathcal O(n^2)$.

## Karatsuba's improvement

In the early 1960s, Anatoly Karatsuba noticed that one of the sum of products in the recursive formula can be rewritten. Specifically:

$$
\begin{align*}
ad+bc & = ad+bc+(ac-ac)+(bd-bd) && (\text{add couple of zeros, no harm done}) \\
      & = ad+ac+bc+bd-ac-bd && (\text{reorder the terms}) \\
      & = a(d+c) +b(c+d)-ac-bd && (\text{factorize the products}) \\
      & = (a+b)(c+d)-ac-bd && (\text{factorize again})
\end{align*}
$$

Using this conversion, the recursive product can be rewritten:

$$
\begin{align*}
xy & = ac10^{2m} + (ad+bc)10^m + bd \\
   & = ac10^{2m} + ((a+b)(c+d)-ac-bd)10^m + bd
\end{align*}
$$

In this form, there are only three products to compute $ac$, $bd$, and $(a+b)(c+d)$. The rest of the operations are additions (and subtractions) which are trivial. The recursive pattern with Karatsuba's substitution becomes $T(n)=3T(n/2)+n$. Here also $r>c^d$ and the multiplication requires $\mathcal O(n^{\log_cr})$. With $c=2$ and $r=3$ the time requirement is $\mathcal O(n^{1.58})$. This is a substantial improvement over $n^2$. To demonstrate the improvement, consider the multiplication of two numbers with $n=1024$ digits. Using the simple verion, with four recursive calls, the multiplication will take about 1,048,576 steps. With Karatsuba's improvement it will require $59,049$ steps.

## Assignment

Using `simple_recursive_multiplication` as a starting point, write a method to multiply two numbers, represented as strings, using the Karatsuba improvement. For simple arithmetic operations (additions and substractions) you may convert strings to integers, perform the operation, and return the result to string form. For example,

```python
difference = str(int("1234")-int("234"))
```
should produce the string `'1000'`.

Use the testing code from earlier to ensure that your Karatsuba multiplications work correctly.

Then conduct side-by-side comparisons measuring how long it takes to multiply numbers with the simple technique and the Karatsuba technique. Comment on your finding for different sized numbers ranging in lengths from $n=4$ to $n=2048$, doubling the size each time (to maintain a power-of-2 number of digits).

Also discuss how to modify the code so that it can handle strings of different length and strings whose lengths are not powers-of-2.

(To time code performance, consider using Python's [`time` module](https://docs.python.org/3/library/time.html)).

### What to submit:

* **Either** a Jupyter notebook with your code, your results, and your observations 
* **or** a `.py` file with your code and a PDF file with your results and observations. You may also provide a link to a GitHub repository containing your work.

If you chose the `py`/PDF option, you must upload your files on Sakai. If you are submitting a Jupyter notebook, it's better to provide a link to it (paste it in Sakai) instead of uploading an `.ipynb` file. If you prefer to work on a language other than Python, please upload to Sakai a file with your source code and a PDF with your results and observations (links to GitHub repositories also acceptable instead of uploads.)